In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
!pip install pronto
# --- Thư viện mới cho xử lý hậu kỳ ---
from pronto import Ontology
from joblib import Parallel, delayed

# ==========================================
# 0. CẤU HÌNH VÀ ĐƯỜNG DẪN (CONFIGURATION)
# ==========================================

CONFIG = {
    'num_classes': 1500,
    'input_dim': 1024,
    'test_split': 0.15,
    'learning_rate': 1e-3, # <--- THAY ĐỔI: 1e-2 thường quá cao, 1e-3 là lựa chọn an toàn hơn
    'epochs': 15,          # <--- THAY ĐỔI: Tăng số epochs để model học được nhiều hơn
    'batch_size': 256,     # <--- THAY ĐỔI: Batch size lớn hơn thường giúp huấn luyện ổn định hơn
    'dropout_rate': 0.3,   # <--- THAY ĐỔI: Tăng dropout một chút để chống overfitting
    'score_threshold': 0.05
}

# ĐƯỜNG DẪN TẬP TRAIN
TRAIN_TERMS_PATH = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv"
TRAIN_IDS_PATH = '/kaggle/input/t5embeds/train_ids.npy'
TRAIN_EMBEDS_PATH = '/kaggle/input/t5embeds/train_embeds.npy'

# ĐƯỜNG DẪN TẬP TEST
TEST_IDS_PATH = '/kaggle/input/t5embeds/test_ids.npy'
TEST_EMBEDS_PATH = '/kaggle/input/t5embeds/test_embeds.npy'

# --- ĐƯỜNG DẪN MỚI CHO XỬ LÝ HẬU KỲ ---
GO_OBO_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
HOMOLOGY_PATH = '/kaggle/input/foldseek-cafa/foldseek_submission.tsv'

# ==========================================
# 1. XỬ LÝ DỮ LIỆU VÀ CĂN CHỈNH (DATA PREPARATION & ALIGNMENT)
# ==========================================
# (Giữ nguyên, phần này đã rất tốt)
print("--- BƯỚC 1: Xử lý Dữ liệu & Căn chỉnh X, Y ---")
train_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t")
labels_to_keep = train_terms['term'].value_counts().index[:CONFIG['num_classes']].tolist()
train_terms_updated = train_terms.loc[train_terms['term'].isin(labels_to_keep)]
y_train_matrix = pd.crosstab(train_terms_updated['EntryID'], train_terms_updated['term'])
labels_final = y_train_matrix.columns.tolist()

train_protein_ids = np.load(TRAIN_IDS_PATH, allow_pickle=True)
train_embeddings = np.load(TRAIN_EMBEDS_PATH)
X_df_raw = pd.DataFrame(train_embeddings, index=train_protein_ids)
X_df_raw.index.name = 'EntryID'

common_ids = list(set(y_train_matrix.index) & set(X_df_raw.index))
Y_train_matrix_aligned = y_train_matrix.loc[common_ids].sort_index()
X_df_raw_aligned = X_df_raw.loc[common_ids].sort_index()

X_train_final = X_df_raw_aligned.values
Y_train_final = Y_train_matrix_aligned.values
print(f"X_train_final shape: {X_train_final.shape}, Y_train_final shape: {Y_train_final.shape}")

# ==========================================
# 2. ĐỊNH NGHĨA VÀ HUẤN LUYỆN MÔ HÌNH (MODEL & TRAINING)
# ==========================================
# (Giữ nguyên)
def build_cafa_model(input_dim, num_classes, dropout_rate, learning_rate):
    model = Sequential([
        tf.keras.Input(shape=(input_dim,)),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(num_classes, activation=None) 
    ])
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['binary_accuracy'] # <--- THAY ĐỔI: 'accuracy' không phù hợp cho multi-label
    )
    return model

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_final, Y_train_final, test_size=CONFIG['test_split'], random_state=42
)

cafa_model = build_cafa_model(
    CONFIG['input_dim'], CONFIG['num_classes'], CONFIG['dropout_rate'], CONFIG['learning_rate']
)

print("\n--- BƯỚC 2: Bắt đầu Huấn luyện Keras MLP ---")
history = cafa_model.fit(
    X_train, Y_train,
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    validation_data=(X_val, Y_val),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)
print("✅ Huấn luyện hoàn thành.")

# ==========================================
# 3. DỰ ĐOÁN VÀ SUBMISSION (INFERENCE & FORMATTING)
# ==========================================

print("\n--- BƯỚC 3: Dự đoán và Tạo Submission ---")

# 3.1. Tải Dữ liệu Test (X_test)
X_test_ids = np.load(TEST_IDS_PATH, allow_pickle=True)
X_test_embeds = np.load(TEST_EMBEDS_PATH)

# 3.2. Dự đoán
test_predictions_logits = cafa_model.predict(X_test_embeds, batch_size=CONFIG['batch_size'])
test_predictions_proba = tf.keras.activations.sigmoid(test_predictions_logits).numpy()

# 3.3. Chuyển đổi sang Định dạng Submission Dài
submission_list = []
threshold = CONFIG['score_threshold']

print("Bắt đầu Chuyển đổi Định dạng và Lọc Ngưỡng...")
for i in tqdm(range(len(X_test_ids)), desc="Formatting Predictions"):
    protein_id = X_test_ids[i]
    predicted_indices = np.where(test_predictions_proba[i] >= threshold)[0]
    for idx in predicted_indices:
        go_term = labels_final[idx]
        score = test_predictions_proba[i, idx]
        submission_list.append([protein_id, go_term, score])

df_predictions = pd.DataFrame(submission_list, columns=['ProteinID', 'GOTermID', 'Confidence'])
print(f"Generated {len(df_predictions)} predictions from T5 model.")

# <--- THÊM MỚI: BƯỚC 3.4 - KẾT HỢP VỚI HOMOLOGY (FOLDSEEK) ---
print("\n--- BƯỚC 3.4: Kết hợp với Homology (Foldseek) ---")
if os.path.exists(HOMOLOGY_PATH):
    homology_df = pd.read_csv(HOMOLOGY_PATH, sep='\t', header=None, names=['ProteinID', 'GOTermID', 'Confidence'])
    print(f"Loaded {len(homology_df)} predictions from Foldseek.")
    
    # Gộp hai nguồn dự đoán
    combined_df = pd.concat([df_predictions, homology_df], ignore_index=True)
    print(f"Total predictions before deduplication: {len(combined_df)}")
    
    # Sắp xếp theo Confidence và loại bỏ trùng lặp, giữ lại cái có điểm cao nhất
    combined_df = combined_df.sort_values('Confidence', ascending=False).drop_duplicates(subset=['ProteinID', 'GOTermID'], keep='first')
    print(f"Total predictions after deduplication (Max Blending): {len(combined_df)}")
else:
    print("⚠️ Homology file not found. Proceeding with T5 model predictions only.")
    combined_df = df_predictions



2025-12-12 09:06:53.817540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765530414.097399      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765530414.176413      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.5 MB/s eta 0:00:00
--- BƯỚC 1: Xử lý Dữ liệu & Căn chỉnh X, Y ---
X_train_final shape: (73756, 1024), Y_train_final shape: (73756, 1500)


2025-12-12 09:07:44.896768: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)



--- BƯỚC 2: Bắt đầu Huấn luyện Keras MLP ---
Epoch 1/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - binary_accuracy: 0.9484 - loss: 0.4814 - val_binary_accuracy: 0.9970 - val_loss: 0.0256
Epoch 2/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - binary_accuracy: 0.9969 - loss: 0.0211 - val_binary_accuracy: 0.9970 - val_loss: 0.0165
Epoch 3/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - binary_accuracy: 0.9970 - loss: 0.0166 - val_binary_accuracy: 0.9970 - val_loss: 0.0154
Epoch 4/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - binary_accuracy: 0.9970 - loss: 0.0159 - val_binary_accuracy: 0.9970 - val_loss: 0.0148
Epoch 5/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - binary_accuracy: 0.9970 - loss: 0.0153 - val_binary_accuracy: 0.9970 - val_loss: 0.0145
Epoch 6/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - binary_accuracy: 0.9970 - loss: 0.0147 - val_binary_accuracy: 0.9970 - val_loss: 0.0141
Epoch 7/15
245/245 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - binary_accuracy: 0.9970 - loss: 0.0146 -

Formatting Predictions: 100%|██████████| 141865/141865 [00:04<00:00, 33409.97it/s]


Generated 1423536 predictions from T5 model.

--- BƯỚC 3.4: Kết hợp với Homology (Foldseek) ---
Loaded 13818804 predictions from Foldseek.
Total predictions before deduplication: 15242340
Total predictions after deduplication (Max Blending): 14579948


In [2]:
# ==========================================
# 3. DỰ ĐOÁN VÀ SUBMISSION (INFERENCE & FORMATTING)
# ==========================================
def process_chunk(df_chunk, ancestor_map):
    """
    Hàm này xử lý một phần nhỏ của DataFrame.
    Nó được định nghĩa ở cấp cao nhất (global) để có thể "pickle".
    """
    propagated_predictions = {}
    # Sử dụng itertuples() nhanh hơn nhiều so với iterrows()
    for row in df_chunk.itertuples(index=False):
        protein_id, term_id, confidence = row.ProteinID, row.GOTermID, row.Confidence

        # Thêm dự đoán gốc
        key = (protein_id, term_id)
        if key not in propagated_predictions or confidence > propagated_predictions[key]:
            propagated_predictions[key] = confidence
        
        # Lấy các tổ tiên từ bản đồ đã tính toán trước
        if term_id in ancestor_map:
            for ancestor_id in ancestor_map[term_id]:
                ancestor_key = (protein_id, ancestor_id)
                if ancestor_key not in propagated_predictions or confidence > propagated_predictions[ancestor_key]:
                    propagated_predictions[ancestor_key] = confidence
                    
    return propagated_predictions

# --- BƯỚC 3.5: THỰC THI PHÂN CẤP GO (PHIÊN BẢN TỐI ƯU) ---
print("\n--- BƯỚC 3.5: Thực thi Hierarchy Propagation (Tối ưu hóa) ---")

def apply_hierarchy_enforcement_optimized(df):
    print("Đang tải file go-basic.obo vào bộ nhớ...")
    with open(GO_OBO_PATH, 'rb') as f:
        go = Ontology(f)
    print(f"Đã tải thành công Ontology với {len(go)} terms.")

    unique_terms = df['GOTermID'].unique()
    print(f"Tiền tính toán bản đồ tổ tiên cho {len(unique_terms)} GO terms duy nhất...")

    term_ancestors_map = {}
    for term_id in tqdm(unique_terms, desc="Building Ancestor Map"):
        try:
            term = go[term_id]
            ancestors = {ancestor.id for ancestor in term.superclasses() if ancestor.id != term_id}
            term_ancestors_map[term_id] = ancestors
        except KeyError:
            term_ancestors_map[term_id] = set()
    print("Bản đồ tổ tiên đã được tạo.")

    n_jobs = os.cpu_count()
    print(f"Chia dữ liệu thành {n_jobs} phần và xử lý song song...")
    df_chunks = np.array_split(df, n_jobs)
    
    parallel = Parallel(n_jobs=n_jobs, backend='multiprocessing')
    # Bây giờ chúng ta gọi hàm global `process_chunk` và truyền `term_ancestors_map` vào
    results_list = parallel(delayed(process_chunk)(chunk, term_ancestors_map) for chunk in tqdm(df_chunks, desc="Parallel Processing"))

    print("Tổng hợp kết quả từ các nhân CPU...")
    final_propagated_predictions = {}
    for chunk_dict in tqdm(results_list, desc="Merging results"):
        for key, value in chunk_dict.items():
            if key not in final_propagated_predictions or value > final_propagated_predictions[key]:
                final_propagated_predictions[key] = value

    final_list = [{'ProteinID': k[0], 'GOTermID': k[1], 'Confidence': v} for k, v in final_propagated_predictions.items()]
    return pd.DataFrame(final_list)

df_submission_final = apply_hierarchy_enforcement_optimized(combined_df)
# -------------------------------------------------------------

# 3.6. Xuất Submission
print("\n--- BƯỚC 3.6: Xuất file Submission ---")
df_submission_final['Confidence'] = df_submission_final['Confidence'].round(4)
# Đổi tên cột để khớp với yêu cầu (nếu cần)
df_submission_final = df_submission_final.rename(columns={'ProteinID': 'EntryID', 'GOTermID': 'GO Term', 'Confidence': 'Score'})

SUBMISSION_FILE_PATH = 'submission.tsv'
df_submission_final[['EntryID', 'GO Term', 'Score']].to_csv(
    SUBMISSION_FILE_PATH, 
    sep='\t', 
    header=False, 
    index=False
)

print(f"\n✅ Đã tạo file submission hoàn thành tại: {SUBMISSION_FILE_PATH}")
print(f"Tổng số dự đoán trong file submission: {len(df_submission_final)}")
print("Top 5 dòng của file cuối cùng:")
print(df_submission_final.head())


--- BƯỚC 3.5: Thực thi Hierarchy Propagation (Tối ưu hóa) ---
Đang tải file go-basic.obo vào bộ nhớ...
Đã tải thành công Ontology với 48106 terms.
Tiền tính toán bản đồ tổ tiên cho 30743 GO terms duy nhất...


Building Ancestor Map: 100%|██████████| 30743/30743 [00:04<00:00, 6296.99it/s]
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Bản đồ tổ tiên đã được tạo.
Chia dữ liệu thành 4 phần và xử lý song song...


Parallel Processing: 100%|██████████| 4/4 [00:00<00:00,  4.36it/s]


Tổng hợp kết quả từ các nhân CPU...


Merging results: 100%|██████████| 4/4 [00:12<00:00,  3.23s/it]



--- BƯỚC 3.6: Xuất file Submission ---

✅ Đã tạo file submission hoàn thành tại: submission.tsv
Tổng số dự đoán trong file submission: 16010569
Top 5 dòng của file cuối cùng:
      EntryID     GO Term  Score
0  A0A3G2FQK2  GO:0140678    1.0
1  A0A3G2FQK2  GO:0098772    1.0
2  A0A3G2FQK2  GO:0003674    1.0
3      P28548  GO:0043229    1.0
4      P28548  GO:0005575    1.0
